<a href="https://colab.research.google.com/github/JZ76/f1tenth_simulator_two_agents/blob/master/LSTM_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
pip install -U numpy==1.18.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import tensorflow
print(tensorflow.__version__)

2.8.2


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import LSTM, Dense, Input, CuDNNLSTM, Masking, Embedding

import os
import re
import gc

In [3]:
def loading_data(folder):
    check_name = re.compile('^ML')
    datasets = []
    for filename in os.listdir(folder):
        files = os.path.join(folder, filename)
        if re.match(check_name, filename) and os.path.isfile(files):
            datasets.append(np.array(pd.read_csv(files, skiprows=1, header=None).iloc[:, :-1], dtype=float))

    gc.collect()
    return datasets

In [4]:
folder = "/content/drive/MyDrive/datasets"
datasets = loading_data(folder)

In [5]:
training_data = datasets[:30]
testing_data = datasets[30:50]

In [6]:
X = []
y = []
max_seq_len = 0
for x in training_data:
    X.append(x[:, 2:])
    y.append(x[:, 0:2])
    max_seq_len = max(max_seq_len, x.shape[0])

In [7]:
special_value = -100.0

In [9]:
padded_inputs = tensorflow.keras.preprocessing.sequence.pad_sequences(
    X, padding="post", value=special_value
)

In [12]:
padded_outputs = tensorflow.keras.preprocessing.sequence.pad_sequences(
    y, padding="post", value=special_value
)

In [8]:
Xpad = []
ypad = []
for i in range(30):
    Xpad.append(np.full((max_seq_len, 1081), fill_value=special_value))
    ypad.append(np.full((max_seq_len, 2), fill_value=special_value))

for s, x in enumerate(X):
    seq_len = x.shape[0]
    Xpad[s][0:seq_len, :] = x

for s, x in enumerate(y):
    seq_len = x.shape[0]
    ypad[s][0:seq_len, :] = x

In [9]:
training_data = []
X = []
y = []
gc.collect()


200

In [12]:
Xpad = np.asarray(Xpad)
ypad = np.asarray(ypad)

In [14]:
LSTM_model = Sequential([
        Masking(input_shape=(max_seq_len, 1081), mask_value=special_value),
        LSTM(20, return_sequences=True, input_shape=(None, 1081)),
        LSTM(8, return_sequences=True),
        Dense(2)
])


In [15]:
LSTM_model.compile(loss="mean_absolute_error", optimizer="adam", metrics=['mean_absolute_error'])
LSTM_model.summary()
LSTM_model.fit(Xpad, ypad, epochs=2, batch_size=5)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, 3514, 1081)        0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 3514, 20)          88160     
_________________________________________________________________
lstm_3 (LSTM)                (None, 3514, 8)           928       
_________________________________________________________________
dense_1 (Dense)              (None, 3514, 2)           18        
Total params: 89,106
Trainable params: 89,106
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
6/6 [==============================] - 97s 16s/step - loss: 2.8210 - mean_absolute_error: 7.4888
Epoch 2/2
6/6 [==============================] - 96s 16s/step - loss: 2.7112 - mean_absolute_error: 7.1973


In [16]:
testing_X = []
testing_y = []

for x in testing_data:
    testing_X.append(x[:, 2:])
    testing_y.append(x[:, 0:2])
    

In [17]:
padded_inputs = tensorflow.keras.preprocessing.sequence.pad_sequences(
    testing_X, padding="post", value=special_value
)

In [18]:
padded_outputs = tensorflow.keras.preprocessing.sequence.pad_sequences(
    testing_y, padding="post", value=special_value
)

In [32]:
print(padded_outputs)

[[[  16    0]
  [  16    0]
  [  16    0]
  ...
  [-100 -100]
  [-100 -100]
  [-100 -100]]

 [[  16    0]
  [  16    0]
  [  16    0]
  ...
  [-100 -100]
  [-100 -100]
  [-100 -100]]

 [[  12    0]
  [  12    0]
  [  12    0]
  ...
  [-100 -100]
  [-100 -100]
  [-100 -100]]

 ...

 [[  16    0]
  [  16    0]
  [  16    0]
  ...
  [-100 -100]
  [-100 -100]
  [-100 -100]]

 [[  16    0]
  [  16    0]
  [  16    0]
  ...
  [-100 -100]
  [-100 -100]
  [-100 -100]]

 [[  16    0]
  [  16    0]
  [  16    0]
  ...
  [-100 -100]
  [-100 -100]
  [-100 -100]]]


In [19]:
LSTM_model.evaluate(padded_inputs, padded_outputs)

1/1 [==============================] - 5s 5s/step - loss: 2.5648 - mean_absolute_error: 7.1521


[2.564760446548462, 7.152050018310547]

In [34]:
LSTM_model.save("/content/drive/MyDrive/f1tenth")